In [1]:
!pip install kaggle


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

mkdir: cannot create directory ‘/teamspace/studios/this_studio/.kaggle’: File exists


In [3]:
! chmod 600 ~/.kaggle/kaggle.json

In [4]:
! kaggle competitions download -c smai-s-25-section-a-project-phase-2

smai-s-25-section-a-project-phase-2.zip: Skipping, found more recently modified local copy (use --force to force download)


In [5]:
!unzip smai-s-25-section-a-project-phase-2.zip

Archive:  smai-s-25-section-a-project-phase-2.zip
replace images_train/images_train/img_0000.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [6]:
from transformers import AutoModelForImageClassification, AutoImageProcessor

model_path = 'facebook/convnext-large-224-22k-1k'
model = AutoModelForImageClassification.from_pretrained(model_path)
processor = AutoImageProcessor.from_pretrained(model_path)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [7]:
model

ConvNextForImageClassification(
  (convnext): ConvNextModel(
    (embeddings): ConvNextEmbeddings(
      (patch_embeddings): Conv2d(3, 192, kernel_size=(4, 4), stride=(4, 4))
      (layernorm): ConvNextLayerNorm()
    )
    (encoder): ConvNextEncoder(
      (stages): ModuleList(
        (0): ConvNextStage(
          (downsampling_layer): Identity()
          (layers): Sequential(
            (0): ConvNextLayer(
              (dwconv): Conv2d(192, 192, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=192)
              (layernorm): ConvNextLayerNorm()
              (pwconv1): Linear(in_features=192, out_features=768, bias=True)
              (act): GELUActivation()
              (pwconv2): Linear(in_features=768, out_features=192, bias=True)
              (drop_path): Identity()
            )
            (1): ConvNextLayer(
              (dwconv): Conv2d(192, 192, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=192)
              (layernorm): ConvNextLayerNorm()
     

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class AngleRegressor(nn.Module):
    def __init__(self):
        super(AngleRegressor, self).__init__()
        self.in_features = 1536

        self.model = nn.Sequential(
            nn.Linear(self.in_features, 512),
            nn.Tanh(),
            nn.Linear(512, 128),
            nn.Tanh(),
            nn.Linear(128, 2)
        )

    def forward(self, x):
        out = self.model(x)
        # out = F.normalize(out, p=2, dim=1)  # Ensure valid [sin, cos]
        return out


In [9]:
model.classifier = AngleRegressor()

In [10]:
import pandas as pd

train = pd.read_csv('labels_train.csv')
val = pd.read_csv('labels_val.csv')

In [11]:
train = train[(train.angle >= 0) & (train.angle < 360)]

In [12]:
import os
import torch
from torch.utils.data import Dataset
from PIL import Image
import pandas as pd
import math

class AngleDataset(Dataset):
    def __init__(self, csv_path, image_dir, transform=None):
        self.df = pd.read_csv(csv_path)
        self.df = self.df[(self.df['angle'] >= 0) & (self.df['angle'] < 360)]  # Filter outliers
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.image_dir, row['filename'])
        image = Image.open(img_path).convert("RGB")

        # Apply transforms directly
        if self.transform:
            pixel_values = self.transform(image)
        else:
            # Fallback if no transform provided
            pixel_values = image

        angle_deg = float(row['angle'])

        # Return a dictionary instead of a tuple to match your training code
        return {
            "pixel_values": pixel_values,
            "angle": torch.tensor(angle_deg, dtype=torch.float32)
        }

In [13]:
from torchvision import transforms
from torchvision.transforms import (
    Compose,
    Resize,
    ToTensor,
    Normalize,
    ColorJitter,
    RandomApply,
    GaussianBlur,
    RandomGrayscale,
)

# Normalization (assuming `feature_extractor` is defined)
normalize = Normalize(mean=processor.image_mean, std=processor.image_std)

# Training transforms
train_transforms = Compose([
    Resize(224),  # Resize first to preserve spatial ratios
    # --- Angle-Safe Augmentations ---
    RandomApply(
        [ColorJitter(brightness=0.3, contrast=0.3, saturation=0.2, hue=0.05)],  # Mild
        p=0.5,
    ),
    RandomApply([GaussianBlur(kernel_size=3)], p=0.3),  # Slight blur
    RandomGrayscale(p=0.1),  # Rare grayscale
    ToTensor(),
    normalize,
])

# Validation transforms (no augmentations)
val_transforms = Compose([
    Resize(224),
    ToTensor(),
    normalize,
])

In [14]:
train_ds = AngleDataset('labels_train.csv',
                       'images_train/images_train',
                       val_transforms)

val_ds = AngleDataset('labels_val.csv',
                       'images_val/images_val',
                       val_transforms)


In [15]:
import wandb

wandb.login(key='5d0f53cfd15364d3cce1d5e9ca6f631fd831a669')

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /teamspace/studios/this_studio/.netrc
wandb: Currently logged in as: shanh042-310 (shanh042-310-iiit-hyderabad) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [37]:
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
import wandb  # optional

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize wandb
wandb.init(project="angle-prediction", name="convnext-angle=1epoch")


In [38]:
train_loader = DataLoader(train_ds, batch_size=16, shuffle=True, num_workers=4)
val_loader = DataLoader(val_ds, batch_size=16, shuffle=False, num_workers=4)


In [39]:
def mean_absolute_angular_error(pred_sin_cos, target_angles_deg):
    """
    pred_sin_cos: (batch_size, 2) - model outputs: [sin, cos]
    target_angles_deg: (batch_size,) - angles in degrees

    Returns:
        Scalar MAAE
    """
    pred_angles_rad = torch.atan2(pred_sin_cos[:, 0], pred_sin_cos[:, 1])  # arctan2(sin, cos)
    pred_angles_deg = torch.rad2deg(pred_angles_rad) % 360
    target_angles_deg = target_angles_deg % 360

    diff = torch.abs(pred_angles_deg - target_angles_deg)
    angular_error = torch.minimum(diff, 360 - diff)
    return angular_error.mean()


In [40]:
criterion = nn.MSELoss()

In [41]:
def validate(model, val_loader, device):
    model.eval()
    val_loss = 0.0
    val_maae = 0.0

    with torch.no_grad():
        for batch in val_loader:
            pixel_values = batch["pixel_values"].to(device)
            angles = batch["angle"].to(device)

            radians = torch.deg2rad(angles)
            target = torch.stack([torch.sin(radians), torch.cos(radians)], dim=1)

            outputs = model(pixel_values)
            logits = outputs.logits if hasattr(outputs, 'logits') else outputs

            loss = criterion(logits, target)
            val_loss += loss.item()

            maae = mean_absolute_angular_error(logits, angles)
            val_maae += maae.item()

    return val_loss/len(val_loader), val_maae/len(val_loader)

In [42]:
import torch
import wandb
from tqdm import tqdm

from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.nn.utils import clip_grad_norm_
import numpy as np

def train_model(model, train_loader, val_loader, optimizer, device, num_epochs=50, patience=5):
    model.to(device)
    best_val_loss = np.inf
    epochs_no_improve = 0
    early_stop = False

    # Cosine annealing scheduler
    scheduler = CosineAnnealingLR(optimizer, T_max=num_epochs, eta_min=1e-6)

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        batch_losses = []

        for batch_idx, batch in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}")):
            pixel_values = batch["pixel_values"].to(device)
            angles = batch["angle"].to(device)

            radians = torch.deg2rad(angles)
            target = torch.stack([torch.sin(radians), torch.cos(radians)], dim=1)

            outputs = model(pixel_values)
            logits = outputs.logits if hasattr(outputs, 'logits') else outputs

            loss = criterion(logits, target)

            optimizer.zero_grad()
            loss.backward()

            # Gradient clipping
            clip_grad_norm_(model.parameters(), max_norm=1.0)

            optimizer.step()

            train_loss += loss.item()
            batch_losses.append(loss.item())

            # Log every 50 batches
            if batch_idx % 50 == 0:
                wandb.log({
                    "batch_train_loss": np.mean(batch_losses[-50:]),
                    "lr": optimizer.param_groups[0]['lr'],
                    "epoch": epoch + (batch_idx/len(train_loader))
                })

        # Validation
        val_loss, val_maae = validate(model, val_loader, device)
        scheduler.step()

        # Early stopping check
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_no_improve = 0
            torch.save(model.state_dict(), 'best_model.pth')
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                early_stop = True

        print(f"Epoch {epoch+1}:")
        print(f"  Train Loss: {train_loss/len(train_loader):.4f}")
        print(f"  Val Loss: {val_loss:.4f}")
        print(f"  Val MAAE: {val_maae:.2f}°")
        print(f"  LR: {optimizer.param_groups[0]['lr']:.2e}")

        wandb.log({
            "epoch_train_loss": train_loss/len(train_loader),
            "val_loss": val_loss,
            "val_maae": val_maae,
            "lr": optimizer.param_groups[0]['lr'],
            "epoch": epoch + 1
        })

        if early_stop:
            print(f"Early stopping triggered after {epoch+1} epochs")
            break

    # Load best model
    model.load_state_dict(torch.load('best_model.pth'))
    return model

In [ ]:
# Initialize with lower learning rate
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)

# Train with all enhancements
model = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    device=device,
    num_epochs=50,       # Increased from 3
    patience=5           # Early stopping patience
)

Epoch 1:   0%|          | 0/409 [00:00<?, ?it/s]

Epoch 1: 100%|██████████| 409/409 [07:57<00:00,  1.17s/it]


Epoch 1:
  Train Loss: 0.0199
  Val Loss: 0.1689
  Val MAAE: 26.01°
  LR: 1.00e-06


In [44]:
wandb.finish()

batch_train_loss,▁██▅▃▄▁▅▇
epoch,▁▂▃▄▄▅▆▇██
epoch_train_loss,▁
lr,█████████▁
val_loss,▁
val_maae,▁
batch_train_loss,0.02192
epoch,1
epoch_train_loss,0.0199
lr,0.0
val_loss,0.16893


In [45]:
def get_angle(logits, round_=False):
    pred_sin_cos = logits
    pred_angles_rad = torch.atan2(pred_sin_cos[0], pred_sin_cos[1])  # arctan2(sin, cos)
    pred_angles_deg = torch.rad2deg(pred_angles_rad) % 360
    pred_angles_deg = pred_angles_deg % 360
    
    return pred_angles_deg

In [46]:
val_loader = DataLoader(val_ds, batch_size=4)

In [47]:
len(val_ds)

369

In [48]:
preds = []
preds_round = []
for batch in tqdm(val_loader):
    torch.cuda.empty_cache()
    pixel_values = batch["pixel_values"].to(device)

    logits = model(pixel_values).logits
    for logit in logits:
        angle = get_angle(logit)
        preds.append(angle.item())
        preds_round.append(round(angle.item()))

100%|██████████| 93/93 [00:12<00:00,  7.58it/s]


In [49]:
len(preds)

369

In [50]:
!unzip images_test.zip

Archive:  images_test.zip
replace images_test/img_0000.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [51]:
test_transforms = Compose([
    Resize(224),
    ToTensor(),
    normalize
])

In [52]:
test_df = val.copy()
test_df['filename'] = os.listdir('images_test')

test_df.to_csv('labels_test.csv', index=False)

In [53]:

test_ds = AngleDataset('labels_test.csv', 'images_test', test_transforms)

test_loader = DataLoader(test_ds, batch_size=4)


for batch in tqdm(test_loader):
    torch.cuda.empty_cache()
    pixel_values = batch["pixel_values"].to(device)

    logits = model(pixel_values).logits
    for logit in logits:
        angle = get_angle(logit)
        preds.append(angle.item())
        preds_round.append(round(angle.item()))

100%|██████████| 93/93 [00:12<00:00,  7.63it/s]


In [34]:
len(preds)

738

In [54]:
pd.DataFrame({'angle': preds}).to_csv('angles_normal_epoch.csv')
pd.DataFrame({'angle': preds_round}).to_csv('angles_rounded.csv')

In [ ]:
len(val_ds), len(test_ds)